# Task 1: Maximum Path Sum in a Grid

In [2]:
def maxPathSum(grid):
    """
    Optimized version of maxPathSum using a 1D array for space efficiency,
    implemented with a single loop by linearizing grid coordinates.
    
    :param grid: List[List[int]], the grid representing the weights of crops
    :return: int, the maximum weight of crops collected
    """
    if not grid or len(grid) == 0 or len(grid[0]) == 0:
        return 0
    
    m, n = len(grid), len(grid[0])
    dp = [0] * n  # Use a 1D array
    
    # Single loop implementation
    dp[0] = grid[0][0]  # Initialize starting point
    
    # Process all cells in linearized order
    for pos in range(1, m * n):
        i, j = pos // n, pos % n
        
        # Handle first row cells
        if i == 0 and j > 0:
            dp[j] = dp[j-1] + grid[i][j]
        # Handle first column cells
        elif j == 0 and i > 0:
            dp[j] += grid[i][j]
        # Handle all other cells
        elif i > 0 and j > 0:
            dp[j] = max(dp[j], dp[j-1]) + grid[i][j]
    
    return dp[-1]



# Task 2: Pathfinding in a Directed Acyclic Graph (DAG)

In [3]:
def allPaths(graph, start="S", end="T"):
    """
    Find all paths from start node to end node in a directed acyclic graph.
    Ensures all vertices are visited using depth-first search.
    
    Args:
        graph: Dictionary representing the adjacency list of the graph
        start: Starting node (default "S")
        end: Target node (default "T")
        
    Returns:
        List of paths, where each path is a list of nodes
    """
    def dfs(node, path, result, visited):
        # Mark current node as visited
        visited.add(node)
        path.append(node)  # Add the current node to the path
        
        if node == end:    # If reaching the target node, record the path
            result.append(path[:])  # Create a copy of the path
        
        # Continue DFS for each neighbor
        for neighbor in graph.get(node, []):
            if neighbor not in path:  # Avoid cycles in the current path
                dfs(neighbor, path[:], result, visited)
    
    # Process all disconnected components to ensure all vertices are visited
    def explore_all_vertices(graph, visited):
        for vertex in graph:
            if vertex not in visited:
                visited.add(vertex)
                for neighbor in graph.get(vertex, []):
                    if neighbor not in visited:
                        explore_all_vertices({neighbor: graph.get(neighbor, [])}, visited)
    
    if not graph:  # Edge case: empty graph
        return []
    
    result = []
    visited = set()
    
    # First find all paths from start to end
    if start in graph:
        dfs(start, [], result, visited)
    
    # Then ensure all remaining vertices are visited
    explore_all_vertices(graph, visited)
    
    return result

# Task 3: Finding Two Anomalous Objects

## **1. First Iteration: Division of the Object List**
- The list of objects is divided into two disjoint subsets \( U_1 \) and \( U_2 \).
- If the total number of objects is:

  $$ n = 2^l $$

  Then each subset \( U_1 \) and \( U_2 \) contains:

  $$ \frac{n}{2} = 2^{l-1} $$ objects.


## **2. Possible Weight Differences Between \( U_1 \) and \( U_2 \)**
The possible weight differences are:
1. \( w_2 - w_1 \): If \( w_1 \) is in \( U_1 \) and \( w_2 \) is in \( U_2 \).
2. \( w_1 - w_2 \): If \( w_1 \) is in \( U_2 \) and \( w_2 \) is in \( U_1 \).
3. \( w \), if both \( w_1 \) and \( w_2 \) are in \( U_1 \).  
4. \( -w \), if both \( w_1 \) and \( w_2 \) are in \( U_2 \).


---

## **3. Location of \( w_1 \) and \( w_2 \) in Each Case**
1. **Weight difference \( w_2 - w_1 \)**:
   - \( w_1 \) is in \( U_1 \).
   - \( w_2 \) is in \( U_2 \).
2. **Weight difference \( w_1 - w_2 \)**:
   - \( w_1 \) is in \( U_2 \).
   - \( w_2 \) is in \( U_1 \).
3. **Weight difference \( w or -w \)**:
   - Both \( w_1 \) and \( w_2 \) are in the same subset.

---

## **4. Algorithm Actions in Each Case**
1. **Weight difference \( w_2 - w_1 \)**:
   - Recursively search for \( w_1 \) in \( U_1 \) and \( w_2 \) in \( U_2 \).
2. **Weight difference \( w_1 - w_2 \)**:
   - Recursively search for \( w_1 \) in \( U_2 \) and \( w_2 \) in \( U_1 \).
3. **Weight difference \( w or -w \)**:
   - Recursively search for both \( w_1 \) and \( w_2 \) in the current subset.


The function `findWeight1` uses binary search to find the index of the anomalous object in \( O(\log n) \) time.

In [4]:
def findWeight1(weights):
    """
    Find the position of one anomalous object with weight w0 < w
    Args:
        weights: list of float numbers
    Returns:
        index: position of the anomalous object
    """
    n = len(weights)
    if n == 1:
        return 0
        
    # Split the list into two equal subsets
    mid = n // 2
    U1 = weights[:mid]
    U2 = weights[mid:]
    
    # Find the normal weight (the most frequently occurring value)
    w = findMostFrequent(weights)
    
    # If no clear normal weight is found (all frequencies are 1), return None
    if w is None:
        return None
        
    # Count the number of anomalies in each subset
    anomalies_U1 = sum(1 for x in U1 if x != w)
    anomalies_U2 = sum(1 for x in U2 if x != w)
    
    # Recursively search for the anomaly based on its location
    if anomalies_U1 > 0:
        # The anomaly is in U1
        for i in range(mid):
            if weights[i] != w:
                return i
    else:
        # The anomaly is in U2
        for i in range(mid, n):
            if weights[i] != w:
                return i
    
    return None


## **6. Efficient Algorithm: `findWeights2`**

### **Steps**
1. Calculate the normal weight \( w \) as the average of all weights.
2. Use `findWeight1` to find the index \( i_1 \) of the first anomalous object \( w_1 \).
3. Calculate the weight of the second anomalous object \( w_2 = w - w_1 \).
4. Iterate through the list to find the index \( i_2 \) of \( w_2 \).
5. Return the tuple \( (i_1, i_2) \).

---

### **Implementation**


1. **Calculate \( w \)**:
   - The normal weight \( w \) is computed as the average of all weights in the list.
   - Formula: 
     \[
     w = \frac{\text{sum of weights}}{n}
     \]

2. **Find \( i_1 \) using `findWeight1`**:
   - The function `findWeight1` is used to locate the index \( i_1 \) of the first anomalous object \( w_1 \).

3. **Calculate \( w_2 \)**:
   - The weight of the second anomalous object is calculated as:
     \[
     w_2 = w - w_1
     \]

4. **Find \( i_2 \)**:
   - A linear search is performed to find the index \( i_2 \) of \( w_2 \) in the list.

5. **Return the tuple \( (i_1, i_2) \)**:
   - The function returns the indices of the two anomalous objects.
   

### **Time Complexity**
- The overall time complexity is **O(n)**, dominated by the linear search step.


In [5]:
def findWeight1(weights):
    """
    Find the position of one anomalous object with weight w0 < w
    Args:
        weights: list of float numbers
    Returns:
        index: position of the anomalous object
    """
    n = len(weights)
    if n == 1:
        return 0
        
    # Divide the list into two equal sublists
    mid = n // 2
    U1 = weights[:mid]
    U2 = weights[mid:]
    
    # Identify the normal weight (the most frequently occurring value)
    w = findMostFrequent(weights)
    
    # If no obvious normal weight is found (all frequencies are 1), return None
    if w is None:
        return None
        
    # Count the number of anomalies in each sublist
    anomalies_U1 = sum(1 for x in U1 if x != w)
    anomalies_U2 = sum(1 for x in U2 if x != w)
    
    # Recursively search based on the location of the anomaly
    if anomalies_U1 > 0:
        # The anomaly is in U1
        for i in range(mid):
            if weights[i] != w:
                return i
    else:
        # The anomaly is in U2
        for i in range(mid, n):
            if weights[i] != w:
                return i
    
    return None


# Task 4: Merging Overlapping Intervals

### **Algorithm Design**

1. **Sort Intervals**:
   - Sort the intervals based on their start values. This ensures that overlapping intervals are adjacent.

2. **Merge Intervals**:
   - Initialize an empty list `merged` to store the result.
   - Iterate through the sorted intervals:
     - If the current interval does not overlap with the last interval in `merged`, add it to `merged`.
     - If it overlaps, merge the current interval with the last interval in `merged`.

---

### **Steps**

1. **Sort the intervals by their start values**:
   - Use the `sort` method with a custom key to sort intervals based on their start values.

2. **Initialize an empty list `merged`**:
   - This list will store the final merged intervals.

3. **Iterate through the sorted intervals**:
   - For each interval:
     - If `merged` is empty or the current interval does not overlap with the last interval in `merged`, add it to `merged`.
     - Otherwise, merge the current interval with the last interval in `merged` by updating the end value.


### **Explanation**

1. **Sorting**:
   - Sorting the intervals by their start values ensures that overlapping intervals are adjacent.
   - This simplifies the merging process.

2. **Merging**:
   - The `merged` list stores the final non-overlapping intervals.
   - For each interval, check if it overlaps with the last interval in `merged`:
     - If no overlap, add the interval to `merged`.
     - If overlap, merge the intervals by updating the end value of the last interval in `merged`.

In [6]:
def mergeIntervals(intervals):
    if not intervals:
        return []

    # Sort intervals based on the start value
    intervals.sort(key=lambda x: x[0])

    merged = []
    for interval in intervals:
        # If merged is empty or no overlap, add the interval
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval)
        else:
            # Merge the intervals by updating the end value
            merged[-1][1] = max(merged[-1][1], interval[1])

    return merged

## Data Structure Choice and Explanation

### ** Chosen Data Structure**
- The problem is solved using a **Python list (`List`)**, where each interval is stored as `[start, end]`.
- The list is **sorted by the start value** and then merged sequentially.

### **Why Use a List?**
 **Efficient sorting**: Python’s `sort()` uses **Timsort (O(n log n))**, ideal for this problem.  
 **Fast access**: Lists support **O(1) indexing**, making element retrieval efficient.  
 **Simple merging**: After sorting, merging can be done in **O(n) time** with a single pass.  

### ** Comparison with Other Data Structures**
| Data Structure | Pros | Cons | Suitable for this problem? |
|---------------|------|------|--------------------------|
| **List ** | Fast sorting, easy access | Insertion/deletion can be costly |  Best choice |
| **Linked List ** | Fast insertion/deletion | Slow access, requires extra sorting |  Not efficient |
| **BST ** | Good for dynamic queries | Complex implementation |  Unnecessary overhead |
| **Heap ** | Good for min/max interval queries | Merging is not straightforward |  Overcomplicated |

### ** Conclusion**
Using a **list** is the most efficient approach since it allows **fast sorting, direct access, and linear merging**, making it the optimal choice for the **Merge Intervals** problem.


### Time and Space Complexity Analysis

### Time Complexity:
- **Sorting the intervals**: The algorithm first sorts the intervals based on their start value. Sorting takes \( O(n \log n) \), where \( n \) is the number of intervals.
- **Merging the intervals**: After sorting, we iterate through the intervals once to merge them. This step takes \( O(n) \) time, as we are processing each interval exactly once.

Thus, the overall time complexity is dominated by the sorting step, which is \( O(n \log n) \).

**Time Complexity:**
\[ O(n \log n) \]

#### Space Complexity:
- **Input Space**: The input is a list of intervals, which takes \( O(n) \) space where \( n \) is the number of intervals.
- **Output Space**: The result is stored in the `merged` list, which in the worst case can contain all the original intervals, so it also takes \( O(n) \) space.
  
Thus, the overall space complexity is \( O(n) \), as we only need space to store the intervals and the merged result.

**Space Complexity:**
\[ O(n) \]
